In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from pathlib import Path
import regex as re
from openpyxl import load_workbook

In [2]:
path_desc = Path('data/features-desc.csv')
path_ev = Path('data/EV_Charging_Stations_Jan312023.xlsx')
path_geo = Path('data/uscities.xlsx')

In [3]:
desc = pd.read_csv(path_desc)
desc.head()

,name,desc,null_values
0,fuel_type_code,The type of alternative fuel the station provi...,0
1,station_name,The name of the station.,2
2,street_address,The street address of the station's location.,34
3,intersection_directions,Brief additional information about how to loca...,51600
4,city,The city of the station's location.,0


In [4]:
def describe(n):
    try:
        return desc.loc[desc['name']==n, 'desc'].to_numpy()[0]
    except (IndexError, KeyError) as e:
        return None

In [37]:
data = pd.read_excel(path_ev, sheet_name='Raw', engine='openpyxl')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54238 entries, 0 to 54237
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Fuel Type Code                    54238 non-null  object        
 1   Station Name                      54236 non-null  object        
 2   Street Address                    54204 non-null  object        
 3   Intersection Directions           2637 non-null   object        
 4   City                              54238 non-null  object        
 5   State                             54236 non-null  object        
 6   ZIP                               54238 non-null  object        
 7   Plus4                             0 non-null      float64       
 8   Station Phone                     50826 non-null  object        
 9   Status Code                       54238 non-null  object        
 10  Expected Date                     0 non-null  

#### NOTE:
Many empty columns not related to electric vehicles

In [39]:
data = data.rename(lambda x: x.lower().replace('-', '').replace(' ', '_'), axis=1)
data.head()

,fuel_type_code,station_name,street_address,intersection_directions,city,state,zip,plus4,station_phone,status_code,...,cng_fill_type_code,cng_psi,cng_vehicle_class,lng_vehicle_class,ev_onsite_renewable_source,restricted_access,rd_blends,rd_blends_(french),rd_blended_with_biodiesel,rd_maximum_biodiesel_level
0,ELEC,Alaska Electric Light & Power,5601 Tonsgard Ct,NaN,Juneau,AK,99801,NaN,907-780-2222,E,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,ELEC,Marine Parking Garage,292 Marine Way,NaN,Juneau,AK,99801,NaN,907-586-5226,E,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,ELEC,Juneau Hydropower,8585 Old Dairy Hwy,NaN,Juneau,AK,99801,NaN,907-789-2775,E,...,NaN,NaN,NaN,NaN,HYDRO,0.0,NaN,NaN,NaN,NaN
3,ELEC,CHUGACH SOUTH CHUGACH EV,5601 Electron Dr,NaN,Anchorage,AK,99518,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELEC,Whistle Hill - Tesla Destination,43540 Kleeb Loop,NaN,Soldotna,AK,99669,NaN,907-398-2931 877-798-3752,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
data.columns

Index(['fuel_type_code', 'station_name', 'street_address',
       'intersection_directions', 'city', 'state', 'zip', 'plus4',
       'station_phone', 'status_code', 'expected_date',
       'groups_with_access_code', 'access_days_time', 'cards_accepted',
       'bd_blends', 'ng_fill_type_code', 'ng_psi', 'ev_level1_evse_num',
       'ev_level2_evse_num', 'ev_dc_fast_count', 'ev_other_info', 'ev_network',
       'ev_network_web', 'geocode_status', 'latitude', 'longitude',
       'date_last_confirmed', 'id', 'updated_at', 'owner_type_code',
       'federal_agency_id', 'federal_agency_name', 'open_date',
       'hydrogen_status_link', 'ng_vehicle_class', 'lpg_primary',
       'e85_blender_pump', 'ev_connector_types', 'country',
       'intersection_directions_(french)', 'access_days_time_(french)',
       'bd_blends_(french)', 'groups_with_access_code_(french)',
       'hydrogen_is_retail', 'access_code', 'access_detail_code',
       'federal_agency_code', 'facility_type', 'cng_dispenser_n

In [41]:
empty = data.loc[:, data.isna().all()]

In [42]:
data = data.drop(columns=empty.columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54238 entries, 0 to 54237
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   fuel_type_code                    54238 non-null  object        
 1   station_name                      54236 non-null  object        
 2   street_address                    54204 non-null  object        
 3   intersection_directions           2637 non-null   object        
 4   city                              54238 non-null  object        
 5   state                             54236 non-null  object        
 6   zip                               54238 non-null  object        
 7   station_phone                     50826 non-null  object        
 8   status_code                       54238 non-null  object        
 9   groups_with_access_code           54238 non-null  object        
 10  access_days_time                  45802 non-nu

In [43]:
geo = pd.read_excel(path_geo, engine='openpyxl')

In [44]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28338 entries, 0 to 28337
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          28338 non-null  object 
 1   city_ascii    28338 non-null  object 
 2   state_id      28338 non-null  object 
 3   state_name    28338 non-null  object 
 4   county_fips   28338 non-null  int64  
 5   county_name   28338 non-null  object 
 6   lat           28338 non-null  float64
 7   lng           28338 non-null  float64
 8   population    28338 non-null  int64  
 9   density       28338 non-null  int64  
 10  source        28338 non-null  object 
 11  military      28338 non-null  bool   
 12  incorporated  28338 non-null  bool   
 13  timezone      28338 non-null  object 
 14  ranking       28338 non-null  int64  
 15  zips          28337 non-null  object 
 16  id            28338 non-null  int64  
dtypes: bool(2), float64(2), int64(5), object(8)
memory usage: 3.3+ MB


### Examining sparse columns

In [45]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(desc.sort_values('null_values', ascending=False))

,name,desc,null_values
15,ev_other_info,NaN,54188
36,ev_on-site_renewable_source,NaN,53877
12,ev_level1_evse_num,"For electric stations, the number of Level 1 EVSE ports.",53538
26,federal_agency_name,NaN,53187
25,federal_agency_id,NaN,53187
33,federal_agency_code,NaN,53187
3,intersection_directions,Brief additional information about how to locate the station.,51600
11,cards_accepted,A space-separated list of payment methods accepted. Payment methods are given as code values as described below:,50260
32,access_detail_code,"A description of other station access information, given as code values as described below:",49684
14,ev_dc_fast_count,NaN,47264


### NOTE:

Many features still contain null values and other undesirables are still present. Now need to go through each and decide whether the null values can be filled or should be dropped.

Of the **filled/mostly-filled** columns (<10000 null), **keeping**: *latitude, longitude, city, zip, status_code, geocode_status, id, fuel_type_code, access_code, station_name, ev_network, open_date, ev_connector_types, access_days_time, ev_level2_evse_num, date_last_confirmed*

**dropping**:
<ul>
    <li><em>groups_with_access_code</em>: deprecated</li>
    <li><em>groups_with_access_code_(french)</em>: deprecated, french</li>
    <li><em>city, state, country</em>: geo table will provide all info from <em>zip</em></li>
    <li><em>updated_at</em>: not necessary</li>
    <li><em>fuel_type_code</em>: dataset only includes ELEC charging stations</li>
    <li><em>street_address</em>: geo table will provide info</li>
    <li><em>station_phone</em>: not necessary</li>
</ul>
    
Of the **other columns**, **keeping**: *owner_type_code (useful), facility_type (potentially useful), ev_pricing (useful), restricted_access (useful), ev_dc_fast_count (None => not DC Fast charger?), access_detail_code (access information; useful), ev_level1_evse_num (None => not lvl 1?), ev_on-site_renewable_source(NaN => None; a valid value), federal_agency_code/id/name (None => not federally owned?, interesting data)*

**dropping**:
<ul>
    <li><em>cards_accepted</em>: not necessary</li>
    <li><em>intersection_directions</em>: not necessary</li>
    <li><em>ev_other_info</em>: only 50 non-null values</li>

In [47]:
drop = ['groups_with_access_code','groups_with_access_code_(french)','city','state','country','updated_at','fuel_type_code','street_address','station_phone','cards_accepted','intersection_directions', 'ev_other_info']

In [48]:
data = data.drop(columns=drop)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54238 entries, 0 to 54237
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   station_name                54236 non-null  object        
 1   zip                         54238 non-null  object        
 2   status_code                 54238 non-null  object        
 3   access_days_time            45802 non-null  object        
 4   ev_level1_evse_num          700 non-null    float64       
 5   ev_level2_evse_num          48014 non-null  float64       
 6   ev_dc_fast_count            6974 non-null   float64       
 7   ev_network                  54236 non-null  object        
 8   ev_network_web              45117 non-null  object        
 9   geocode_status              54238 non-null  object        
 10  latitude                    54238 non-null  float64       
 11  longitude                   54238 non-null  float64   

In [51]:
print(list(data.columns))

['station_name', 'zip', 'status_code', 'access_days_time', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_count', 'ev_network', 'ev_network_web', 'geocode_status', 'latitude', 'longitude', 'date_last_confirmed', 'id', 'owner_type_code', 'federal_agency_id', 'federal_agency_name', 'open_date', 'ev_connector_types', 'access_code', 'access_detail_code', 'federal_agency_code', 'facility_type', 'ev_pricing', 'ev_onsite_renewable_source', 'restricted_access']


In [63]:
lvl1 = data.loc[~data['ev_level1_evse_num'].isna(), ['id']]
lvl2 = data.loc[~data['ev_level2_evse_num'].isna(), ['id']]
lvl3 = data.loc[~data['ev_dc_fast_count'].isna(), ['id']]

In [64]:
l12 = lvl1.isin(lvl2).sum()
l13 = lvl1.isin(lvl3).sum()
l23 = lvl2.isin(lvl3).sum()
print(f'1 and 2: {l12}, \n1 and 3: {l13}, \n2 and 3: {l23}')

1 and 2: id    219
dtype: int64, 
1 and 3: id    7
dtype: int64, 
2 and 3: id    1234
dtype: int64


In [65]:
total_w_ports = (lvl1.shape[0] + lvl2.shape[0] + lvl3.shape[0] - (l12+l13+l23))[0]
print(f'{total_w_ports} out of {data.shape[0]} stations have charger levels associated with them')

54228 out of 54238 stations have charger levels associated with them


In [66]:
unknown_chargers = data.loc[data['ev_level1_evse_num'].isna() & data['ev_level2_evse_num'].isna() & data['ev_dc_fast_count'].isna(), :]

In [67]:
charge_cols = ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_count']

In [68]:
# Charger level count: changing nan to 0
data[charge_cols] = data[charge_cols].fillna(0)
# No info available on chargers here
data.loc[unknown_chargers.index, charge_cols] = np.nan

In [69]:
data.loc[unknown_chargers.index, :]

,station_name,zip,status_code,access_days_time,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,ev_network,ev_network_web,geocode_status,...,federal_agency_name,open_date,ev_connector_types,access_code,access_detail_code,federal_agency_code,facility_type,ev_pricing,ev_on-site_renewable_source,restricted_access
1636,Bristol Farms,91030,E,24 hours daily,NaN,NaN,NaN,Non-Networked,NaN,200-9,...,NaN,1997-08-31,NaN,public,NaN,NaN,GROCERY,NaN,NaN,0.0
1653,Japan Center Garage,94115,E,24 hours daily; pay lot,NaN,NaN,NaN,Non-Networked,NaN,GPS,...,NaN,2002-01-31,NaN,public,CREDIT_CARD_ALWAYS,NaN,PARKING_GARAGE,NaN,NaN,0.0
1708,Vallejo Ferry Terminal,94590,E,24 hours daily,NaN,NaN,NaN,Non-Networked,NaN,200-8,...,NaN,2005-07-15,NaN,public,NaN,NaN,TRAVEL_CENTER,Free,NaN,0.0
2388,City of Sacramento - Department of Utilities,95822,E,NaN,NaN,NaN,NaN,Non-Networked,NaN,200-9,...,NaN,2017-04-01,NaN,private,NaN,NaN,MUNI_GOV,NaN,NaN,NaN
18030,CDPS CBI Arvada Lab,80002,E,NaN,NaN,NaN,NaN,NaN,NaN,200-8,...,NaN,2020-07-06,NaN,private,NaN,NaN,NaN,NaN,NaN,NaN
31510,DTE Energy - Macomb Service Center,48038,E,24 hours daily,NaN,NaN,NaN,Non-Networked,NaN,GPS,...,NaN,2010-01-01,NaN,public,NaN,NaN,UTILITY,Free,NaN,0.0
42919,Forest Service - Umpqua National Forest,97443,E,USDA fleet use only,NaN,NaN,NaN,NaN,NaN,200-9,...,U.S. Department of Agriculture,2013-06-30,NaN,private,NaN,USDA,NATL_PARK,NaN,NaN,NaN
